# Heat Exchanger NTU Unit Model with Aqueous MEA System

![](heat_exchanger_4.svg)

**Problem Statement**: In this example, we will be transfering heat from hot to cold streams of MEA (monoethylamine), simulating heat integration of an aqueous MEA system.

**Hot Side Inlet**

Flow Rate = 60.54879 mol/s

Mole fraction (CO2) = 0.0158

Mole fraction (H2O) = 0.8747

Mole fraction (MEA) = 0.1095

Pressure = 202650 Pa

Temperature = 392.23 K

**Cold Side Inlet**

Flow Rate = 63.01910 mol/s

Mole fraction (CO2) = 0.0414

Mole fraction (H2O) = 0.8509

Mole fraction (MEA) = 0.1077

Pressure = 202650 Pa

Temperature = 326.36 K

This example will demonstrate the simulation of the NTU heat exchanger by fixing the following degrees of freedom:
- heat transfer area
- heat transfer coefficient
- effectiveness
- hot and cold side pressure changes


IDAES documentation reference for heat exchanger NTU model: https://idaes-pse.readthedocs.io/en/latest/reference_guides/model_libraries/generic/unit_models/heat_exchanger_ntu.html

This example utilizes the NTU heat exchanger for a shell and tube system.

## Importing necessary tools

First, import the required IDAES and Pyomo modules:

In [ ]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the MEA property package to create a property block for the flowsheet
from idaes.models_extra.column_models.properties.MEA_solvent \
    import configuration as aqueous_mea

from idaes.models.properties.modular_properties.base.generic_property import (
        GenericParameterBlock)

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the default IPOPT solver
from idaes.core.solvers import get_solver

# Import a heat exchanger unit
from idaes.models.unit_models.heat_exchanger_ntu import HeatExchangerNTU as HXNTU

## Setting up the flowsheet
Then, we will create the `ConcreteModel` foundation, attach the steady state flowsheet, and declare the property parameter block that will used for the hot and cold sides.

More information on this general workflow can be found here: https://idaes-pse.readthedocs.io/en/stable/how_to_guides/workflow/general.html

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

m.fs.hotside_properties = GenericParameterBlock(**aqueous_mea)

m.fs.coldside_properties = GenericParameterBlock(**aqueous_mea)

Next, we specify hot and cold streams; note that heat flow from the hot to the cold stream is calculated using the transfer unit method for the `HXNTU` unit model.

The unit is created below:

In [ ]:
# Create an instance of the heat exchanger unit, attaching it to the flowsheet
# Specify that the property package to be used with the heat exchanger is the one we created earlier.
m.fs.heat_exchanger = HXNTU(
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": m.fs.hotside_properties,
           "has_pressure_change": True},
    tube={"property_package": m.fs.coldside_properties,
          "has_pressure_change": True})

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

In [ ]:
assert DOF_initial == 15

## Fixing input specifications
For this problem, we will fix the inlet conditions, re-evaluate the degrees of freedom to ensure the problem is square (i.e. DOF=0), and run two different options for unit specifications:

In [ ]:
m.fs.heat_exchanger.hot_side_inlet.flow_mol[0].fix(60.54879) # mol/s
m.fs.heat_exchanger.hot_side_inlet.temperature[0].fix(392.23) # K
m.fs.heat_exchanger.hot_side_inlet.pressure[0].fix(202650) # Pa
m.fs.heat_exchanger.hot_side_inlet.mole_frac_comp[0, "CO2"].fix(0.0158) # dimensionless
m.fs.heat_exchanger.hot_side_inlet.mole_frac_comp[0, "H2O"].fix(0.8747) # dimensionless
m.fs.heat_exchanger.hot_side_inlet.mole_frac_comp[0, "MEA"].fix(0.1095) # dimensionless

m.fs.heat_exchanger.cold_side_inlet.flow_mol[0].fix(63.01910) # mol/s
m.fs.heat_exchanger.cold_side_inlet.temperature[0].fix(326.36) # K
m.fs.heat_exchanger.cold_side_inlet.pressure[0].fix(202650) # Pa
m.fs.heat_exchanger.cold_side_inlet.mole_frac_comp[0, "CO2"].fix(0.0414) # dimensionless
m.fs.heat_exchanger.cold_side_inlet.mole_frac_comp[0, "H2O"].fix(0.8509) # dimensionless
m.fs.heat_exchanger.cold_side_inlet.mole_frac_comp[0, "MEA"].fix(0.1077) # dimensionless

DOF_initial = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_initial))

### Option 1: Fix heat transfer coefficient (HTC) and dimensions of each domain
Below, we fix the heat exchanger area, effectiveness factor and heat transfer coefficient, which yields a fully defined problem with zero degrees of freedom that may be initialized and solved:

In [ ]:
# Unit design variables
m.fs.heat_exchanger.area.fix(100) # m2
m.fs.heat_exchanger.heat_transfer_coefficient.fix(200) # W/m2/K
m.fs.heat_exchanger.effectiveness.fix(0.7) # dimensionless

m.fs.heat_exchanger.hot_side.deltaP.fix(-2000) # Pa
m.fs.heat_exchanger.cold_side.deltaP.fix(-2000) # Pa

DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
assert DOF_final == 0

Now that the problem is square (zero degress of freedom), we can initialize and solve the full model:

In [ ]:
# Initialize the flowsheet, and set the output at INFO
m.fs.heat_exchanger.initialize(outlvl=idaeslog.INFO)

# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = get_solver()
solve_status = opt.solve(m, tee = True)

# Display a readable report
m.fs.heat_exchanger.report()

In [ ]:
from pyomo.environ import assert_optimal_termination
import pytest

# Check if termination condition is optimal
assert_optimal_termination(solve_status)

assert value(m.fs.heat_exchanger.hot_side_outlet.temperature[0]) == pytest.approx(344.00, abs=1e-2)
assert value(m.fs.heat_exchanger.cold_side_outlet.temperature[0]) == pytest.approx(374.33, abs=1e-2)

### Option 2: Unfix shell length  and fix shell outlet temperatures

In the previous case, we fixed the heat exchanger area and overall heat transfer coefficient. However, given that the models in IDAES are equation oriented, we can fix the outlet variables. For example, we can fix the hot outlet temperature and heat exchanger length, and solve for the heat exchanger area and effectiveness factor that will satisfy that condition.


In [ ]:
m.fs.heat_exchanger.effectiveness.unfix()
m.fs.heat_exchanger.area.unfix()
m.fs.heat_exchanger.hot_side_outlet.temperature.fix(344.0)

DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
result = opt.solve(m)

print(result)

# Display a readable report
m.fs.heat_exchanger.report()

In [ ]:
# Check if termination condition is optimal
assert_optimal_termination(result)

assert value(m.fs.heat_exchanger.area) == pytest.approx(100, abs=1e-2)
assert value(m.fs.heat_exchanger.effectiveness[0]) == pytest.approx(0.7, abs=1e-2)
assert value(m.fs.heat_exchanger.cold_side_outlet.temperature[0]) == pytest.approx(
    374.33, rel=1e-3)